# Research Ideas

In [11]:
"""
You are a Professional Research Advisor who work for Research Boost. You help users with research ideas by asking questions to build context, then search PubMed and then respond with a structured response. 
You ask insightful yet conversational questions to probe the user’s ideas, knowledge, and observations. Your role is to help connect abstract thoughts to concrete research ideas while ensuring the user feels supported and engaged in the process. 

<First Step>
Your task is to lightly interview the user;

Start with an easy, open question that invites the user to share a thought or observation. Build on their response with follow-up questions. 
Ask one question per turn. 
Don't ask more than 5 questions in total.
Avoid overly technical or complex questions initially; keep them conversational and engaging.
Let the conversation flow naturally to build trust and comfort.

<Second Step>
After you have enough context and know the relevant keywords, Search PubMed for research articles using them. 

<Third Step>
After you get the PubMed response, respond to the user with 5 new research ideas after critically analyzing the PubMed response.
Also add the names of the articles as sources at the end of the response.

<Forth Step>
Generate a Concept Map(Mermaid Diagram) that visualizes various research directions for the current context. This map will group ideas into several major themes and breaks down each theme into subtopics.
"""

"\nYou are a Professional Research Advisor who work for Research Boost. You help users with research ideas by asking questions to build context, then search PubMed and then respond with a structured response. \nYou ask insightful yet conversational questions to probe the user’s ideas, knowledge, and observations. Your role is to help connect abstract thoughts to concrete research ideas while ensuring the user feels supported and engaged in the process. \n\n<First Step>\nYour task is to lightly interview the user;\n\nStart with an easy, open question that invites the user to share a thought or observation. Build on their response with follow-up questions. \nAsk one question per turn. \nDon't ask more than 5 questions in total.\nAvoid overly technical or complex questions initially; keep them conversational and engaging.\nLet the conversation flow naturally to build trust and comfort.\n\n<Second Step>\nAfter you have enough context and know the relevant keywords, Search PubMed for resear

In [ ]:
from openai import OpenAI
import json
from dotenv import load_dotenv
import os
from pymed import PubMed

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("API key not found. Check your .env file.")

client = OpenAI(api_key=api_key)

def pubmed_search(query, max_results=5):
    pubmed = PubMed(tool="MyTool", email="sharozhaseeb1@gmail.com")
    results = pubmed.query(query, max_results=max_results)

    result_list = []
    for article in results:
        child_dict = {}
        child_dict['title'] = article.title
        child_dict['authors'] = [((author['lastname'] or '') + ', ' + (author['firstname'] or '')).strip(', ') for author in article.authors]
        child_dict['abstract'] = article.abstract
        child_dict['publication_date'] = article.publication_date
        result_list.append(child_dict)
    
    return result_list

tools = [{
    "type": "function",
    "function": {
        "name": "pubmed_search",
        "description": "Searches PubMed for research articles based on a given query.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "The keyword to search for."
                },
                "max_results": {
                    "type": "integer",
                    "description": "The maximum number of results to retrieve.",
                    "default": 5
                }
            },
            "required": ["query"]
        }
    }
}]

messages = [{"role": "system", "content": "You are a Professional Research Advisor who work for Research Boost. You help users with research ideas by asking questions to build context, then search PubMed and then respond with a structured response. \nYou ask insightful yet conversational questions to probe the user’s ideas, knowledge, and observations. Your role is to help connect abstract thoughts to concrete research ideas while ensuring the user feels supported and engaged in the process. \n\n<First Step>\nYour task is to lightly interview the user;\n\nStart with an easy, open question that invites the user to share a thought or observation. Build on their response with follow-up questions. \nAsk one question per turn. \nDon't ask more than 5 questions in total.\nAvoid overly technical or complex questions initially; keep them conversational and engaging.\nLet the conversation flow naturally to build trust and comfort.\n\n<Second Step>\nAfter you have enough context and know the relevant keywords, Search PubMed for research articles using them. \n\n<Third Step>\nAfter you get the PubMed response, respond to the user with 5 new research ideas after critically analyzing the PubMed response.\nAlso add the names of the articles as sources at the end of the response.\n\n<Forth Step>\nGenerate a Concept Map(Mermaid Diagram) that visualizes various research directions for the current context. This map will group ideas into several major themes and breaks down each theme into subtopics."},
            {"role": "user", "content": "I work in machine learning specifically transformers, can you get me some research ideas?"}]
            # {"role": "user", "content": "Hi, who are you ?"}]


completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
    # tool_choice="auto"
)


In [2]:
print(completion.choices[0])

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_euOxD1SrpSINfZ6O8fzSWs5J', function=Function(arguments='{"query":"transformers machine learning","max_results":5}', name='pubmed_search'), type='function')], annotations=[]))


In [3]:
completion.choices[0].finish_reason

#output for tool call will be 'tool_calls'
#output for chat will be 'stop'

'tool_calls'

In [4]:
tool_calls = completion.choices[0].message.tool_calls

for tool_call in tool_calls:
    print(tool_call.to_dict())



{'id': 'call_euOxD1SrpSINfZ6O8fzSWs5J', 'function': {'arguments': '{"query":"transformers machine learning","max_results":5}', 'name': 'pubmed_search'}, 'type': 'function'}


In [5]:
tool_call = completion.choices[0].message.tool_calls[0]
args = json.loads(tool_call.function.arguments)

result = pubmed_search(args["query"], args["max_results"])

In [6]:
print(result)

[{'title': 'Accelerating the inference of string generation-based chemical reaction models for industrial applications.', 'authors': ['Andronov, Mikhail', 'Andronova, Natalia', 'Wand, Michael', 'Schmidhuber, Jürgen', 'Clevert, Djork-Arné'], 'abstract': "Transformer-based, template-free SMILES-to-SMILES translation models for reaction prediction and single-step retrosynthesis are of interest to computer-aided synthesis planning systems, as they offer state-of-the-art accuracy. However, their slow inference speed limits their practical utility in such applications. To address this challenge, we propose speculative decoding with a simple chemically specific drafting strategy and apply it to the Molecular Transformer, an encoder-decoder transformer for conditional SMILES generation. Our approach achieves over 3X faster inference in reaction product prediction and single-step retrosynthesis with no loss in accuracy, increasing the potential of the transformer as the backbone of synthesis pl

In [7]:
from datetime import date


def convert_pubmed_resp_to_str(articles):
    master_str = ""
    for article in articles:
        formatted_string = f"""Title: {article['title']}
        Authors: {', '.join(article['authors'])}
        Abstract: {article['abstract']}
        Publication Date: {article['publication_date'].strftime('%Y-%m-%d')}
        {'-' * 80}\n"""
        master_str += formatted_string
    return str(master_str)



In [8]:
results_str = convert_pubmed_resp_to_str(result)

In [9]:
messages.append(completion.choices[0].message)  # append model's function call message
messages.append({                               # append result message
    "role": "tool",
    "tool_call_id": tool_call.id,
    "content": results_str
})

completion_2 = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)

In [10]:
print(completion_2.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Here is a summary of recent research articles related to transformers in machine learning:\n\n1. **Accelerating the Inference of String Generation-Based Chemical Reaction Models for Industrial Applications**\n   - **Authors**: Andronov, Mikhail; Andronova, Natalia; Wand, Michael; Schmidhuber, Jürgen; Clevert, Djork-Arné\n   - **Abstract**: This study focuses on improving the inference speed of transformer-based SMILES-to-SMILES translation models used for reaction prediction and retrosynthesis. The authors introduce speculative decoding and Speculative Beam Search to achieve over 3 times faster inference without sacrificing accuracy. This advancement enhances the practical utility of transformers in synthesis planning systems.\n   - **Publication Date**: March 11, 2025\n\n2. **Principal Component Analysis and Fine-Tuned Vision Transformation Integrating Model Explainability for Breast Cancer Pre

In [ ]:
import subprocess
import shutil

def install_mermaid_cli():
    """
    Checks if mermaid-cli (mmdc) is installed. If not, installs it using npm.
    """
    if shutil.which("mmdc") is None:
        print("Mermaid-cli (mmdc) not found. Installing...")
        try:
            subprocess.run(["npm", "install", "-g", "@mermaid-js/mermaid-cli"], check=True)
            print("Mermaid-cli installed successfully!")
        except subprocess.CalledProcessError as e:
            print(f"Error installing mermaid-cli: {e}")
            return False
    else:
        print("Mermaid-cli is already installed.")
    return True

def mermaid_to_svg(input_mmd: str, output_svg: str):
    """
    Converts a Mermaid diagram (.mmd) into an SVG file using the mermaid-cli (mmdc).
    Installs mermaid-cli if not found.

    :param input_mmd: Path to the input .mmd file.
    :param output_svg: Path to save the output .svg file.
    """
    if not install_mermaid_cli():
        print("Mermaid-cli installation failed. Exiting.")
        return

    try:
        # Run the mmdc command to generate the SVG
        subprocess.run(["mmdc", "-i", input_mmd, "-o", output_svg], check=True)
        print(f"Successfully converted {input_mmd} to {output_svg}")
    except subprocess.CalledProcessError as e:
        print(f"Error during conversion: {e}")

# Example usage
mermaid_to_svg("diagram.mmd", "output.svg")


In [ ]:
# vas = {'id': 'call_euOxD1SrpSINfZ6O8fzSWs5J', 'function': {'arguments': '{"query":"transformers machine learning","max_results":5}', 'name': 'pubmed_search'}, 'type': 'function'}

# print(vas['function']['name'])

pubmed_search


In [4]:
vas = [{"d":2}]
l = []
vas.extend(l)
print(vas)

[{'d': 2}]


In [5]:
for i in vas:
    print(type(i))

<class 'dict'>
